### Connect to localhosted DB and DW (postgreSQL)

In [20]:
import pandas as pd
import psycopg2
from psycopg2 import sql

## Setting credentials to access database

In [231]:
PGHOST = 'localhost'
PGDATABASE = 'SP703db'
PGUSER = '*****' # hiding sensitive info
PGPASSWORD = '*****' # hiding sensitive info

## Establish a database connection

In [232]:
def create_connect():
    dbconn = psycopg2.connect(
        dbname=PGDATABASE,
        user=PGUSER,
        password=PGPASSWORD,
        host=PGHOST,
        port='5432'
    )
    return dbconn

# Extract and Load to Database

## Patient data

In [4]:
# source of patient data
patientdf = pd.read_excel('Patientsdata.xlsx')

#### Inserting Patients data into Patients table in SP703db

In [5]:
#creating connection
dbconn = create_connect()

# Create a cursor
cur = dbconn.cursor()

# Insert data from csv file to Patient table in SP703 db
for item, row in patientdf.iterrows():
    cur.execute(
        '''INSERT INTO patients (PatientID, FirstName, LastName, DateOfBirth, ContactNumber, Address) 
           VALUES (%s, %s, %s, %s, %s, %s) ON CONFLICT (PatientID) DO NOTHING''',
        (row['PatientID'], row['FirstName'], row['LastName'], 
         row['DateOfBirth'], row['ContactNumber'], row['Address']))
    

# commit the changes
dbconn.commit()

# Execute SQL queries to check if we successfully inserted the data
cur.execute('SELECT * FROM patients')
patientdb_tbl = pd.DataFrame(cur.fetchall(), columns=patientdf.columns)

# Close the cursor and connection
cur.close()
dbconn.close()

In [6]:
# checking if we successfully inserted the data in db table
patientdb_tbl

,PatientID,FirstName,LastName,DateOfBirth,ContactNumber,Address
0,1,John,Doe,1985-05-15,123-456-7890,123 Main St
1,2,Jane,Smith,1990-08-20,987-654-3210,456 Elm St
2,3,David,Johnson,1998-03-10,555-555-5555,789 Oak Ave
3,4,Emily,Williams,1980-11-28,111-222-3333,321 Pine Rd
4,5,Michael,Brown,1975-07-12,444-777-8888,555 Cedar Ln
5,6,Susan,Anderson,1982-09-05,222-333-4444,789 Maple St
6,7,Robert,White,1993-04-30,666-999-1111,234 Birch Ave
7,8,Linda,Miller,1977-12-15,888-333-2222,567 Oak St
8,9,William,Moore,1989-06-22,444-666-7777,890 Pine Rd
9,10,Mary,Johnson,1987-03-08,777-888-9999,456 Elm St


## Providers data

In [7]:
# source of Providers data
providerdf = pd.read_excel('Providersdata.xlsx')

#### Inserting Providers data into Providers table in SP703db

In [66]:
#creating connection
dbconn = create_connect()

# Create a cursor
cur = dbconn.cursor()

# Insert data from csv file to Providers table in SP703 db
for item, row in providerdf.iterrows():
    cur.execute(
        '''INSERT INTO providers (ProviderID, FirstName, LastName, Specialization, 
            LicenseNumber, ContactNumber, Email) 
           VALUES (%s, %s, %s, %s, %s, %s, %s) ON CONFLICT (ProviderID) DO NOTHING''',
        (row['ProviderID'], row['FirstName'], row['LastName'], 
         row['Specialization'], row['LicenseNumber'], row['ContactNumber'],
         row['Email']))
    

# commit the changes
dbconn.commit()

# Execute SQL queries to check if we successfully inserted the data
cur.execute('SELECT * FROM providers')
providerdb_tbl = pd.DataFrame(cur.fetchall(), columns=providerdf.columns)

# Close the cursor and connection
cur.close()
dbconn.close()

In [67]:
# checking if we successfully inserted the data in db table
providerdb_tbl

,ProviderID,FirstName,LastName,Specialization,LicenseNumber,ContactNumber,Email
0,1,Dr. Sarah,Anderson,Cardiology,MD123456,555-123-7890,sarah.anderson@example.com
1,2,Dr. Robert,Clark,Orthopedics,MD789012,444-987-6543,robert.clark@example.com
2,3,Dr. Lisa,Smith,Internal Medicine,MD345678,777-333-2222,lisa.smith@example.com
3,4,Dr. John,Wilson,Neurology,MD234567,888-555-9999,john.wilson@example.com
4,5,Dr. Emily,Jones,Dermatology,MD567890,999-111-2222,emily.jones@example.com


## Appointments data

In [10]:
# source of Appointments data
appointmentdf = pd.read_excel('Appointmentsdata.xlsx')

#### Inserting Appointments data into Appointments table in SP703db

In [64]:
#creating connection
dbconn = create_connect()

# Create a cursor
cur = dbconn.cursor()

# Insert data from csv file to Appointments table in SP703 db
for item, row in appointmentdf.iterrows():
    cur.execute(
        '''INSERT INTO appointments (AppointmentID, AppointmentDate, AppointmentTime, PatientID, 
                                      ProviderID, AppointmentType, AppointmentStatus) 
           VALUES (%s, %s, %s, %s, %s, %s, %s) ON CONFLICT (AppointmentID) DO NOTHING''',
        (row['AppointmentID'], row['AppointmentDate'], row['AppointmentTime'], 
         row['PatientID'], row['ProviderID'], row['AppointmentType'],
         row['AppointmentStatus']))
    

# commit the changes
dbconn.commit()

# Execute SQL queries to check if we successfully inserted the data
cur.execute('SELECT * FROM appointments')
appointmentdb_tbl = pd.DataFrame(cur.fetchall(), columns=appointmentdf.columns)

# Close the cursor and connection
cur.close()
dbconn.close()

In [65]:
# checking if we successfully inserted the data in db table
appointmentdb_tbl

,AppointmentID,AppointmentDate,AppointmentTime,PatientID,ProviderID,AppointmentType,AppointmentStatus
0,1,2023-10-10,09:00:00,1,1,General Cardiology Check-up,Scheduled
1,2,2023-10-11,10:30:00,2,2,Orthopedic Assessment,Confirmed
2,3,2023-10-12,14:15:00,3,1,Cardiology Check-up,Scheduled
3,4,2023-10-13,11:45:00,4,4,Neurological Evaluation,Confirmed
4,5,2023-10-14,08:30:00,5,5,Dermatological Examination,Scheduled
5,6,2023-10-15,13:30:00,6,2,Orthopedic Assessment,Confirmed
6,7,2023-10-16,10:15:00,7,5,Dermatological Examination,Scheduled
7,8,2023-10-17,15:45:00,8,1,Cardiology Check-up,Confirmed
8,9,2023-10-18,09:30:00,9,4,Neurological Evaluation,Scheduled
9,10,2023-10-19,11:00:00,10,3,Internal Medicine Check-up,Confirmed


## Procedure data

In [13]:
# source of Procedures data
proceddf = pd.read_excel('Proceduresdata.xlsx')

#### Inserting Procedure data into Procedure table in SP703db

In [62]:
#creating connection
dbconn = create_connect()

# Create a cursor
cur = dbconn.cursor()

# Insert data from csv file to Procedures table in SP703 db
for item, row in proceddf.iterrows():
    cur.execute(
        '''INSERT INTO procedures (ProcedureID, ProcedureName, ProcedureDescription, 
                                   ProcedureCost) 
           VALUES (%s, %s, %s, %s) ON CONFLICT (ProcedureID) DO NOTHING''',
        (row['ProcedureID'], row['ProcedureName'], row['ProcedureDescription'], 
         row['ProcedureCost']))
    

# commit the changes
dbconn.commit()

# Execute SQL queries to check if we successfully inserted the data
cur.execute('SELECT * FROM procedures')
proceddb_tbl = pd.DataFrame(cur.fetchall(), columns=proceddf.columns)

# Close the cursor and connection
cur.close()
dbconn.close()

In [63]:
# checking if we successfully inserted the data in db table
proceddb_tbl

,ProcedureID,ProcedureName,ProcedureDescription,ProcedureCost
0,1,Cardiology Check-up,Comprehensive cardiology check-up including EC...,250.00
1,2,Orthopedics Consultation,"Orthopedic assessment and consultation, includ...",150.00
2,3,Internal Medicine Check-up,Comprehensive internal medicine check-up and h...,200.00
3,4,Neurology Consultation,"Neurological evaluation, consultation, and dia...",180.00
4,5,Dermatology Check-up,"Dermatological examination, consultation, and ...",120.00


## AppointmentProcedure data (junction table)

In [16]:
# source of Appointments data
apptproceddf = pd.read_excel('AppointmentProceduredata.xlsx')
apptproceddf = apptproceddf.convert_dtypes(int)

#### Inserting AppointmentProcedure data into AppointmentProcedure table in SP703db

In [60]:
#creating connection
dbconn = create_connect()

# Create a cursor
cur = dbconn.cursor()

# Insert data from csv file to AppointmentProcedure table in SP703 db
for item, row in apptproceddf.iterrows():    
    cur.execute(
        '''INSERT INTO appointmentprocedure (AppointmentProcedureID, AppointmentID, ProcedureID) 
           VALUES (%s, %s, %s) ON CONFLICT (AppointmentProcedureID) DO NOTHING''',
        (row['AppointmentProcedureID'], row['AppointmentID'], row['ProcedureID']))
    

# commit the changes
dbconn.commit()

# Execute SQL queries to check if we successfully inserted the data
cur.execute('SELECT * FROM appointmentprocedure')
apptproceddb_tbl = pd.DataFrame(cur.fetchall(), columns=apptproceddf.columns)

# Close the cursor and connection
cur.close()
dbconn.close()

In [61]:
# checking if we successfully inserted the data in db table
apptproceddb_tbl

,AppointmentProcedureID,AppointmentID,ProcedureID
0,1,1,1
1,2,2,2
2,3,3,1
3,4,4,4
4,5,5,5
5,6,6,2
6,7,7,5
7,8,8,1
8,9,9,4
9,10,10,3


# Extract, Transform, and Load to Data Warehouse

# EXTRACT

#### After loading the data into the database SP703db, we will again extract and transform the data and load it to the SP703dw datawarehouse

## Providers data from SP703db database

In [59]:
#creating connection
dbconn = create_connect()

# Create a cursor
cur = dbconn.cursor()

# Execute SQL queries to retrieved data
cur.execute('SELECT * FROM providers')
providerdf = pd.DataFrame(cur.fetchall(), columns=providerdf.columns)

# Close the cursor and connection
cur.close()
dbconn.close()

#### Result from fetching data from database SP703db
##### This data doesnt need transformation as it has the same structure in the SP703dw data warehouse

In [49]:
providerdf

,ProviderID,FirstName,LastName,Specialization,LicenseNumber,ContactNumber,Email
0,1,Dr. Sarah,Anderson,Cardiology,MD123456,555-123-7890,sarah.anderson@example.com
1,2,Dr. Robert,Clark,Orthopedics,MD789012,444-987-6543,robert.clark@example.com
2,3,Dr. Lisa,Smith,Internal Medicine,MD345678,777-333-2222,lisa.smith@example.com
3,4,Dr. John,Wilson,Neurology,MD234567,888-555-9999,john.wilson@example.com
4,5,Dr. Emily,Jones,Dermatology,MD567890,999-111-2222,emily.jones@example.com


## Appointments data from SP703db database

In [77]:
#creating connection
dbconn = create_connect()

# Create a cursor
cur = dbconn.cursor()

# Execute SQL queries to retrieved data
cur.execute('SELECT appointmentid, appointmentdate FROM appointments')
appointdf = pd.DataFrame(cur.fetchall(), columns=appointmentdb_tbl.columns[:2])

# Close the cursor and connection
cur.close()
dbconn.close()

#### Result from fetching data from database SP703db. 
##### Next is we need to manipulate this data during transformation stage to meet the required structure before loading it to SP703dw data warehouse

In [84]:
# result from fetching data from database SP703db
appointdf

,AppointmentID,AppointmentDate,Date,DayOfWeek,Month,Quarter,Year
0,1,2023-10-10,10,Tuesday,October,4,2023
1,2,2023-10-11,11,Wednesday,October,4,2023
2,3,2023-10-12,12,Thursday,October,4,2023
3,4,2023-10-13,13,Friday,October,4,2023
4,5,2023-10-14,14,Saturday,October,4,2023
5,6,2023-10-15,15,Sunday,October,4,2023
6,7,2023-10-16,16,Monday,October,4,2023
7,8,2023-10-17,17,Tuesday,October,4,2023
8,9,2023-10-18,18,Wednesday,October,4,2023
9,10,2023-10-19,19,Thursday,October,4,2023


## Specialization data  from "other database"

###### Note: We will just assume that this data comes from "other database" within the medical clinic, this is already been discussed in the documentation figure 8 page 11

In [90]:
# this is a list of specialization by the providers, but it will add more if the provider does have two specialization.
# but for now, we will just assume a 1:1 ratio.
spec = pd.read_excel('Specializationdata.xlsx')
spec

,SpecializationID,Specialization
0,1,Cardiology
1,2,Orthopedics
2,3,Internal Medicine
3,4,Neurology
4,5,Dermatology


## Fact Table data from "other database" and SP703db

###### Note: We will just assume that this data comes from "other database" within the medical clinic, this is already been discussed in the documentation figure 8 page 11

In [168]:
spec_dict = {}

# storing the providers id and specialization into a dictionary
for i, r in providerdf.iterrows():
    spec_dict[r['Specialization']] = r['ProviderID']

spec_dict

{'Cardiology': 1,
 'Orthopedics': 2,
 'Internal Medicine': 3,
 'Neurology': 4,
 'Dermatology': 5}

In [167]:
# data for AppointmentCount, AverageSatisfactionRating, AverageAppointmentDuration, and AverageWaitingTime
mixdata = pd.read_excel('Ratingsetcdata.xlsx')
mixdata

,AppointmentCount,AverageSatisfactionRating,AverageAppointmentDuration,AverageWaitingTime
0,1,4.6,30,5
1,1,4.4,40,6
2,1,4.2,25,4
3,1,4.1,45,3
4,1,4.8,35,5
5,1,4.5,38,6
6,1,4.7,42,1
7,1,4.3,28,10
8,1,4.0,50,5
9,1,4.9,33,6


##### Note: Code block below will extract data comes from the SP703db database not from the "other database"

In [233]:
#creating connection
dbconn = create_connect()

# Create a cursor
cur = dbconn.cursor()

# new column names for extracted data
cols = ['TimeID', 'ProviderID']

# Execute SQL queries to retrieved data
cur.execute('SELECT appointmentid, providerid FROM appointments')
patxprovdf = pd.DataFrame(cur.fetchall(), columns=cols)

# Close the cursor and connection
cur.close()
dbconn.close()

In [234]:
patxprovdf

,TimeID,ProviderID
0,1,1
1,2,2
2,3,1
3,4,4
4,5,5
5,6,2
6,7,5
7,8,1
8,9,4
9,10,3


# TRANSFORM

#### We need to transform the Appointment data to follow the structure in Time dimension

In [103]:
# Convert 'AppointmentDate' column to datetime
appointdf['AppointmentDate'] = pd.to_datetime(appointdf['AppointmentDate'])

# Split the 'AppointmentDate' column into separate columns
appointdf['Date'] = appointdf['AppointmentDate'].dt.date
appointdf['DayOfWeek'] = appointdf['AppointmentDate'].dt.day
appointdf['Month'] = appointdf['AppointmentDate'].dt.month
appointdf['Quarter'] = appointdf['AppointmentDate'].dt.quarter
appointdf['Year'] = appointdf['AppointmentDate'].dt.year

# dropping unnecessary column
trfndappointdf = appointdf.drop(columns='AppointmentDate')
trfndappointdf

,AppointmentID,Date,DayOfWeek,Month,Quarter,Year
0,1,2023-10-10,10,10,4,2023
1,2,2023-10-11,11,10,4,2023
2,3,2023-10-12,12,10,4,2023
3,4,2023-10-13,13,10,4,2023
4,5,2023-10-14,14,10,4,2023
5,6,2023-10-15,15,10,4,2023
6,7,2023-10-16,16,10,4,2023
7,8,2023-10-17,17,10,4,2023
8,9,2023-10-18,18,10,4,2023
9,10,2023-10-19,19,10,4,2023


#### Merging the data from SP703 database and "other database" for the Provider Performance Fact Table

In [235]:
# Add a new column 'Specialization' based on 'ProviderID'
patxprovdf['SpecializationID'] = patxprovdf['ProviderID'].replace(spec_dict)

In [236]:
factdw_tbl = pd.concat([patxprovdf, mixdata], axis=1)
factdw_tbl

,TimeID,ProviderID,SpecializationID,AppointmentCount,AverageSatisfactionRating,AverageAppointmentDuration,AverageWaitingTime
0,1,1,1,1,4.6,30,5
1,2,2,2,1,4.4,40,6
2,3,1,1,1,4.2,25,4
3,4,4,4,1,4.1,45,3
4,5,5,5,1,4.8,35,5
5,6,2,2,1,4.5,38,6
6,7,5,5,1,4.7,42,1
7,8,1,1,1,4.3,28,10
8,9,4,4,1,4.0,50,5
9,10,3,3,1,4.9,33,6


# LOAD

## Setting credentials to access datawarehouse

In [238]:
PGHOST = 'localhost'
PGDATAWAREHOUSE = 'SP703dw'
PGUSER = '******' # need to hide sensitive info
PGPASSWORD = '******' # need to hide sensitive info

## Establish a datawarehouse connection

In [239]:
def create_connectdw():
    dwconn = psycopg2.connect(
        dbname=PGDATAWAREHOUSE,
        user=PGUSER,
        password=PGPASSWORD,
        host=PGHOST,
        port='5432'
    )
    return dwconn

## Load the Providers data from SP702 database to Providers Dimension in SP703dw data warehouse

In [240]:
#creating connection
dwconn = create_connectdw()

# Create a cursor
cur = dwconn.cursor()

# Insert data from providers table into SP703 dw provider_dim
for item, row in providerdf.iterrows():
    cur.execute(
        '''INSERT INTO provider_dim (ProviderID, FirstName, LastName, 
                                    Specialization, LicenseNumber, ContactNumber, Email) 
           VALUES (%s, %s, %s, %s, %s, %s, %s) ON CONFLICT (ProviderID) DO NOTHING''',
        (row['ProviderID'], row['FirstName'], row['LastName'], 
         row['Specialization'], row['LicenseNumber'], row['ContactNumber'],
         row['Email']))
    

# commit the changes
dwconn.commit()

# Execute SQL queries to check if we successfully inserted the data
cur.execute('SELECT * FROM provider_dim')
providerdw_tbl = pd.DataFrame(cur.fetchall(), columns=providerdf.columns)

# Close the cursor and connection
cur.close()
dwconn.close()

In [241]:
# to check if we successfully inserted the data to datawarehouse
providerdw_tbl

,ProviderID,FirstName,LastName,Specialization,LicenseNumber,ContactNumber,Email
0,1,Dr. Sarah,Anderson,Cardiology,MD123456,555-123-7890,sarah.anderson@example.com
1,2,Dr. Robert,Clark,Orthopedics,MD789012,444-987-6543,robert.clark@example.com
2,3,Dr. Lisa,Smith,Internal Medicine,MD345678,777-333-2222,lisa.smith@example.com
3,4,Dr. John,Wilson,Neurology,MD234567,888-555-9999,john.wilson@example.com
4,5,Dr. Emily,Jones,Dermatology,MD567890,999-111-2222,emily.jones@example.com


## Load the transformed Appointment data from SP702 database to Time Dimension in SP703dw data warehouse

In [242]:
#creating connection
dwconn = create_connectdw()

# Create a cursor
cur = dwconn.cursor()

# Insert data from transformed Appointment table into SP703 dw time_dim
for item, row in trfndappointdf.iterrows():
    cur.execute(
        '''INSERT INTO time_dim (TimeID, Date, DayOfWeek, Month, Quarter, Year) 
           VALUES (%s, %s, %s, %s, %s, %s) ON CONFLICT (TimeID) DO NOTHING''',
        (row['AppointmentID'], row['Date'], row['DayOfWeek'], 
         row['Month'], row['Quarter'], row['Year']))
    

# commit the changes
dwconn.commit()

#we need to declare new columns
cols = ['TimeID', 'Date', 'DayOfWeek', 'Month', 'Quarter', 'Year']

# Execute SQL queries to check if we successfully inserted the data
cur.execute('SELECT * FROM time_dim')
timedw_tbl = pd.DataFrame(cur.fetchall(), columns=cols)

# Close the cursor and connection
cur.close()
dwconn.close()

In [243]:
# to check if we successfully inserted the data to datawarehouse
timedw_tbl

,TimeID,Date,DayOfWeek,Month,Quarter,Year
0,1,2023-10-10,10,10,4,2023
1,2,2023-10-11,11,10,4,2023
2,3,2023-10-12,12,10,4,2023
3,4,2023-10-13,13,10,4,2023
4,5,2023-10-14,14,10,4,2023
5,6,2023-10-15,15,10,4,2023
6,7,2023-10-16,16,10,4,2023
7,8,2023-10-17,17,10,4,2023
8,9,2023-10-18,18,10,4,2023
9,10,2023-10-19,19,10,4,2023


## Load the Specialization data from "other database" to Specialization Dimension in SP703dw data warehouse

In [244]:
#creating connection
dwconn = create_connectdw()

# Create a cursor
cur = dwconn.cursor()

# Insert data from "other database" table in SP703 dw
for item, row in spec.iterrows():
    cur.execute(
        '''INSERT INTO specialization_dim (SpecializationID, Specialization) 
           VALUES (%s, %s) ON CONFLICT (SpecializationID) DO NOTHING''',
        (row['SpecializationID'], row['Specialization']))
    

# commit the changes
dwconn.commit()

# Execute SQL queries to check if we successfully inserted the data
cur.execute('SELECT * FROM specialization_dim')
specdw_tbl = pd.DataFrame(cur.fetchall(), columns=spec.columns)

# Close the cursor and connection
cur.close()
dwconn.close()

In [245]:
# to check if we successfully inserted the data to datawarehouse
specdw_tbl

,SpecializationID,Specialization
0,1,Cardiology
1,2,Orthopedics
2,3,Internal Medicine
3,4,Neurology
4,5,Dermatology


## Load the  data from SP703db database to Provider Performance Fact  in SP703dw data warehouse

In [258]:
#creating connection
dwconn = create_connectdw()

# Create a cursor
cur = dwconn.cursor()

# getting the latest key
latest_key = 0
cur.execute('SELECT MAX(performanceid) AS key FROM providerperformance_fact')
get_key = pd.DataFrame(cur.fetchall(), columns=['key'])
if get_key['key'].item() == None:
    latest_key = 0
else:
    latest_key = get_key['key'].item()


# Insert data from other database + SP703db table into providerperformance fact table in SP703dw
for item, row in factdw_tbl.iterrows():
    latest_key+=1
    cur.execute(
        '''INSERT INTO providerperformance_fact (PerformanceID, TimeID, ProviderID, SpecializationID,
           AppointmentCount, AverageSatisfactionRating, AverageAppointmentDuration, AverageWaitTime) 
           VALUES (%s, %s, %s, %s, %s, %s, %s, %s) ON CONFLICT (PerformanceID) DO NOTHING''',
        (latest_key, row['TimeID'], row['ProviderID'], row['SpecializationID'], 
         row['AppointmentCount'], row['AverageSatisfactionRating'],
        row['AverageAppointmentDuration'], row['AverageWaitingTime']))
    
# commit the changes
dwconn.commit()

cols = ['PerformanceID', 'TimeID', 'ProviderID', 'SpecializationID', 
        'AppointmentCount', 'AverageSatisfactionRating', 'AverageAppointmentDuration', 'AverageWaitTime']

# Execute SQL queries to check if we successfully inserted the data
cur.execute('SELECT * FROM providerperformance_fact')
specdw_tbl = pd.DataFrame(cur.fetchall(), columns=cols)

# Close the cursor and connection
cur.close()
dwconn.close()

In [259]:
# checking datawarehouse
specdw_tbl

,PerformanceID,TimeID,ProviderID,SpecializationID,AppointmentCount,AverageSatisfactionRating,AverageAppointmentDuration,AverageWaitTime
0,1,1,1,1,1,4.60,30.00,5.00
1,2,2,2,2,1,4.40,40.00,6.00
2,3,3,1,1,1,4.20,25.00,4.00
3,4,4,4,4,1,4.10,45.00,3.00
4,5,5,5,5,1,4.80,35.00,5.00
5,6,6,2,2,1,4.50,38.00,6.00
6,7,7,5,5,1,4.70,42.00,1.00
7,8,8,1,1,1,4.30,28.00,10.00
8,9,9,4,4,1,4.00,50.00,5.00
9,10,10,3,3,1,4.90,33.00,6.00
